# Kaggle Project: Suicide and Depression Detection

**Objective:**

To create a text classifier to detect suicidal intentions on social media.


**Dataset details:**

The dataset is a collection of posts from the "SuicideWatch" and "depression" subreddits of the Reddit platform. The posts are collected using Pushshift API. All posts that were made to "SuicideWatch" from Dec 16, 2008(creation) till Jan 2, 2021, were collected while "depression" posts were collected from Jan 1, 2009, to Jan 2, 2021. All posts collected from SuicideWatch are labeled as suicide, While posts collected from the depression subreddit are labeled as depression. Non-suicide posts are collected from r/teenagers.


**Kaggle source project:**
- https://www.kaggle.com/datasets/nikhileswarkomati/suicide-watch

## Imports

In [ ]:
from libraries.models.bert_app import BERTHuggingFaceClassification
from libraries.utils.statistics import (
    print_classification_metrics,
    print_auc_and_plot_precision_recall,
    df_downsample,
    confidence_sample_size,
    interest_proportion
)
from libraries.utils.dataviz import(
    plot_roc_curve,
    model_evaluation_cm,
    barplot_series,
    evalmetrics_multicategory
)

## Loading dataset

## Model Training

## Model testing